In [1]:
%reload_ext autoreload
%autoreload 2
import os
from pyeed.core import ProteinRecord

##### Here we get all the data

In [2]:
# read in the text file
file_path = 'TEM_Ids.txt'
with open(file_path, 'r') as file:
    lines = file.readlines()

# create a dic with the name and id
# A	TEM-P118	t		AAN05029 	AY130285 	12354869 	view			2be	ESBL	A
# we want to keep the name and the id, these are the second and the fourth columns
# we need to split the line by the tab character
data = {}
count = 0
for line in lines:
    line = line.strip()
    columns = line.split('\t')
    name = columns[1]
    id = columns[4].strip()
    if id == 'EFJ83682':
        continue
    data[name] = id
    count += 1


print('EFJ83682' in list(data.values()))

False


In [6]:
protein_ids_fetched = ProteinRecord.get_ids(list(data.values()))[:20]

# here we want to add the region for the ncbi blast id
# this is a kind of work arround to get the id later for the name mappping
for i in range(0, len(protein_ids_fetched)):
    protein_ids_fetched[i].add_to_regions(
        name="blastquery:" + list(data.keys())[i],
        start=0,
        end=len(protein_ids_fetched[i].sequence)
    )


Output()

Request to 
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=protein&retmode=text&rettype=genbank&id=CCG28759,ADB79
815,CTA52364,QDY98370,AUT06962,QWY17601,AQX83499,AAN05029,QBG79064,CAD24670 failed with status code 429

Rate limit exceeded. Waiting for 1 second...

Request to 
https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=protein&retmode=text&rettype=genbank&id=AMD11804,AAK30
619,AAX56615,AAL77062,AAZ14083,AAS89984,AAG45415,AFC75525,AEK48085,QPG87090 failed with status code 429

Response: {"error":"API rate limit 
exceeded","api-key":"2001:7c0:2049:1d5:fec8:ea3:cc02:667b","count":"4","limit":"3"}

Rate limit exceeded. Waiting for 1 second...

[Region(id='PRK15442', url=None, accession_id=None, name=None, start=2, end=286, json_ld_type=['Region'], json_ld_context={'Region': 'https://github.com/PyEED/pyeed/Region', 'url': 'http://semanticscience.org/resource/SIO_000811', 'accession_id': 'http://semanticscience.org/resource/SIO_000675', 'start': 'http://semanticscience.org/resource/SIO_000943', 'end': 'http://semanticscience.org/resource/SIO_000953'}), Region(id='2dad9608-72fa-4f0c-b396-110d18a0fc46', url=None, accession_id=None, name='blastquery:TEM-1', start=0, end=286, json_ld_type=['Region'], json_ld_context={'Region': 'https://github.com/PyEED/pyeed/Region', 'url': 'http://semanticscience.org/resource/SIO_000811', 'accession_id': 'http://semanticscience.org/resource/SIO_000675', 'start': 'http://semanticscience.org/resource/SIO_000943', 'end': 'http://semanticscience.org/resource/SIO_000953'})]


In [4]:
current_path = os.path.dirname(os.getcwd())
print(len(protein_ids_fetched))

20


In [7]:
# we now search for each protein in the data dic a blast
# then we combine the results im data, we also have to remove double entries
data_proteins = []

"""
for protein in protein_ids_fetched:
    data_proteins = data_proteins + protein.ncbi_blast(n_hits=1000, e_value=0.1)
"""
# no i want to parallelize this
import asyncio

async def fetch_protein(protein):
    return protein.ncbi_blast(n_hits=1000, e_value=0.1)

# create a job for each protein
jobs = [fetch_protein(protein) for protein in protein_ids_fetched]
# run all the jobs
data_proteins = await asyncio.gather(*jobs)


Output()

Output()

Output()

Output()

Output()

...

Output()

Output()

Output()

Output()

Output()

In [ ]:
len(data_proteins[0])

70

In [ ]:
# remove double entries
data_protein_unique = []
data_protein_unique_id = []
for protein in data_proteins:
    if protein.id not in data_protein_unique_id:
        data_protein_unique.append(protein)
        data_protein_unique_id.append(protein.id)

print(len(data_protein_unique))
print('AAP20891' in data_protein_unique_id)
print('CAJ85677' in data_protein_unique_id)

AttributeError: 'list' object has no attribute 'id'

In [ ]:
# save the blast search results
output_folder_blast_search = os.path.join(current_path, "TEM-lactamase", "data", "data_blast_search_big_{}_{}_{}".format('list003', 'all', int(0.01*1000)))
os.makedirs(output_folder_blast_search, exist_ok=True)
counter = 0
for hit in data_proteins:
    with open(output_folder_blast_search + "/{}.json".format(counter), "w") as f:
        f.write(hit.json())
        counter += 1